In [1]:
import pandas as pd

data = pd.read_csv(r'/kaggle/input/haxophone/playerpassingstats.csv')

data = data.rename(columns={"Unnamed: 0": "league", "Unnamed: 1": "ID", "Unnamed: 2": "Club", 
                            "Unnamed: 3": "name"})

data = data.dropna(subset=['name'])

data["ID"] = pd.to_numeric(data["ID"], errors='coerce')

data = data[~data["ID"].isin([1617, 1718])].dropna(subset=['ID'])

data_sorted = data.sort_values(by=["name", "age", "born"])
# data = data_sorted.iloc[:1000] 
print(data.dtypes)


league         object
ID            float64
Club           object
name           object
nation         object
pos            object
age            object
born          float64
90s           float64
Total          object
Total.1        object
Total.2        object
Total.3        object
Total.4        object
Short          object
Short.1        object
Short.2        object
Medium         object
Medium.1       object
Medium.2       object
Long           object
Long.1         object
Long.2         object
Ast           float64
xAG           float64
Expected       object
Expected.1     object
KP            float64
1/3           float64
PPA           float64
CrsPA         float64
PrgP          float64
dtype: object


<ipython-input-1-d6ffc0b5a39c>:3: DtypeWarning: Columns (1,9,10,11,12,13,14,15,16,17,18,19,20,21,22,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'/kaggle/input/haxophone/playerpassingstats.csv')


In [2]:
import pandas as pd

def contains_mostly_numbers(column):
    """Check if a column contains mostly numeric values (including numbers stored as strings)."""
    num_count = sum(str(val).replace('.', '', 1).isdigit() for val in column if pd.notna(val))
    total_count = column.notna().sum()  # Ignore NaN values in count
    return num_count / total_count > 0.5  # Convert if more than 50% of the column is numeric

for col in data.columns:
    if contains_mostly_numbers(data[col]):
        data[col] = pd.to_numeric(data[col], errors='coerce')  # Convert to numbers, keeping NaNs

print(data.dtypes)

league         object
ID            float64
Club           object
name           object
nation         object
pos            object
age           float64
born          float64
90s           float64
Total         float64
Total.1       float64
Total.2       float64
Total.3       float64
Total.4       float64
Short         float64
Short.1       float64
Short.2       float64
Medium        float64
Medium.1      float64
Medium.2      float64
Long          float64
Long.1        float64
Long.2        float64
Ast           float64
xAG           float64
Expected      float64
Expected.1    float64
KP            float64
1/3           float64
PPA           float64
CrsPA         float64
PrgP          float64
dtype: object


In [3]:
import pandas as pd

pd.set_option('future.no_silent_downcasting', True)  # Suppress warning

def fill_missing_values(group):
    for col in group.columns:
        if col != "name":  
            if pd.api.types.is_numeric_dtype(group[col]): 
                mean_value = group[col].mean()
                group[col] = group[col].fillna(mean_value if pd.notna(mean_value) else 0)  # Replace NaN mean with 0
            else:  
                mode_value = group[col].mode()
                group[col] = group[col].fillna(mode_value.iloc[0] if not mode_value.empty else "Unknown")  
    return group

grouped_data = data.groupby(['name'], group_keys=False)  
data_filled = grouped_data.apply(fill_missing_values).reset_index(drop=True)  # Drop extra index

# Check remaining NaNs
nan_counts = data_filled.isna().sum()
print(nan_counts[nan_counts > 0])  

data_filled


Series([], dtype: int64)


<ipython-input-3-0b6e1010f337>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_filled = grouped_data.apply(fill_missing_values).reset_index(drop=True)  # Drop extra index


,league,ID,Club,name,nation,pos,age,born,90s,Total,...,Long.2,Ast,xAG,Expected,Expected.1,KP,1/3,PPA,CrsPA,PrgP
0,ENG-Premier League,1819.0,Arsenal,Aaron Ramsey,WAL,MF,27.0,1990.0,14.8,652.0,...,72.9,6.0,2.0,2.3,4.0,25.0,51.0,16.0,4.0,88.0
1,ENG-Premier League,1819.0,Arsenal,Ainsley Maitland-Niles,ENG,DF,20.0,1997.0,11.0,453.0,...,54.5,1.0,0.9,1.0,0.1,6.0,27.0,14.0,1.0,60.0
2,ENG-Premier League,1819.0,Arsenal,Alex Iwobi,NGA,"FW,MF",22.0,1996.0,21.9,767.0,...,45.7,6.0,7.0,5.6,-1.0,40.0,53.0,65.0,8.0,141.0
3,ENG-Premier League,1819.0,Arsenal,Alexandre Lacazette,FRA,FW,27.0,1991.0,27.8,581.0,...,55.8,8.0,3.7,3.4,4.3,31.0,36.0,39.0,3.0,92.0
4,ENG-Premier League,1819.0,Arsenal,Bernd Leno,GER,GK,26.0,1992.0,31.5,817.0,...,44.8,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16575,ITA-Serie A,2425.0,Venezia,Nicholas Pierini,ITA,MF,0.0,1998.0,0.9,15.0,...,50.0,0.0,0.1,0.1,-0.1,2.0,1.0,0.0,0.0,2.0
16576,ITA-Serie A,2425.0,Venezia,Nunzio Lella,ITA,MF,0.0,2000.0,0.1,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16577,ITA-Serie A,2425.0,Venezia,Richie Sagrado,BEL,DF,0.0,2004.0,1.6,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0
16578,ITA-Serie A,2425.0,Venezia,Ridgeciano Haps,SUR,DF,30.0,1993.0,11.3,361.0,...,47.9,0.0,0.4,0.4,-0.4,5.0,29.0,10.0,5.0,41.0


In [4]:
import pandas as pd

data=data_filled
base_columns = {}
for col in data.columns:
    base = col.split('.')[0] 
    if col == base or not data[col].dtype.kind in 'iuf':  # Skip non-numeric columns
        continue
    base_columns.setdefault(base, []).append(col)
print(base_columns)

columns=list(base_columns.keys())
# Compute the mean for grouped numeric columns
for base, cols in base_columns.items():
    data[base] = data[[base] + cols].mean(axis=1)  # Average base column and its duplicates
    data.drop(columns=cols, inplace=True)  # Remove duplicate columns

nan_counts = data.isna().sum()
print(nan_counts[nan_counts > 0]) 
print(columns)

{'Total': ['Total.1', 'Total.2', 'Total.3', 'Total.4'], 'Short': ['Short.1', 'Short.2'], 'Medium': ['Medium.1', 'Medium.2'], 'Long': ['Long.1', 'Long.2'], 'Expected': ['Expected.1']}
Series([], dtype: int64)
['Total', 'Short', 'Medium', 'Long', 'Expected']


In [5]:
data

,league,ID,Club,name,nation,pos,age,born,90s,Total,...,Medium,Long,Ast,xAG,Expected,KP,1/3,PPA,CrsPA,PrgP
0,ENG-Premier League,1819.0,Arsenal,Aaron Ramsey,WAL,MF,27.0,1990.0,14.8,2676.56,...,167.666667,58.300000,6.0,2.0,3.15,25.0,51.0,16.0,4.0,88.0
1,ENG-Premier League,1819.0,Arsenal,Ainsley Maitland-Niles,ENG,DF,20.0,1997.0,11.0,2098.96,...,147.000000,46.500000,1.0,0.9,0.55,6.0,27.0,14.0,1.0,60.0
2,ENG-Premier League,1819.0,Arsenal,Alex Iwobi,NGA,"FW,MF",22.0,1996.0,21.9,3239.92,...,223.000000,54.566667,6.0,7.0,2.30,40.0,53.0,65.0,8.0,141.0
3,ENG-Premier League,1819.0,Arsenal,Alexandre Lacazette,FRA,FW,27.0,1991.0,27.8,2058.00,...,117.833333,40.933333,8.0,3.7,3.85,31.0,36.0,39.0,3.0,92.0
4,ENG-Premier League,1819.0,Arsenal,Bernd Leno,GER,GK,26.0,1992.0,31.5,7703.58,...,325.433333,258.600000,0.0,0.0,0.00,0.0,3.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16575,ITA-Serie A,2425.0,Venezia,Nicholas Pierini,ITA,MF,0.0,1998.0,0.9,77.00,...,27.333333,17.666667,0.0,0.1,0.00,2.0,1.0,0.0,0.0,2.0
16576,ITA-Serie A,2425.0,Venezia,Nunzio Lella,ITA,MF,0.0,2000.0,0.1,21.60,...,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
16577,ITA-Serie A,2425.0,Venezia,Richie Sagrado,BEL,DF,0.0,2004.0,1.6,107.48,...,31.266667,1.000000,0.0,0.0,0.00,0.0,2.0,0.0,0.0,2.0
16578,ITA-Serie A,2425.0,Venezia,Ridgeciano Haps,SUR,DF,30.0,1993.0,11.3,1849.10,...,158.800000,39.633333,0.0,0.4,0.00,5.0,29.0,10.0,5.0,41.0


In [6]:
import pandas as pd

# Sort data for consistency
data_sorted = data.sort_values(by=["name", "ID"])

def weighted_average(group):
    # Take first occurrence for categorical columns (constant for each player)
    base_info = group.iloc[0][["name", "nation", "pos", "born"]].to_dict()

    # Assign weights based on season recency (newer seasons get higher weight)
    group["weight"] = group["ID"].rank(method="dense", ascending=True)  # Older seasons have lower rank
    group["weight"] = group["weight"] / group["weight"].sum()  # Normalize weights so they sum to 1

    # Compute weighted average for numerical attributes
    weighted_stats = {}
    for col in columns:
        weighted_stats[col] = (group[col] * group["weight"]).sum()

    # Get the most recent Club & League
    most_recent_season = group.loc[group["ID"].idxmax()]
    base_info["Club"] = most_recent_season["Club"]
    base_info["league"] = most_recent_season["league"]
    base_info["ID"] = most_recent_season["ID"]  # Keep the latest season's ID for reference

    return {**base_info, **weighted_stats}

# Apply function to group by player name
final_df = pd.DataFrame(data_sorted.groupby("name").apply(weighted_average).tolist())

final_df

<ipython-input-6-67ceb9d8bbb0>:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = pd.DataFrame(data_sorted.groupby("name").apply(weighted_average).tolist())


,name,nation,pos,born,Club,league,ID,Total,Short,Medium,Long,Expected
0,Aaron Ciammaglichella,ITA,MF,2005.0,Torino,ITA-Serie A,2425.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Aaron Connolly,IRL,FW,2000.0,Brighton,ENG-Premier League,2021.0,295.740000,68.611111,42.222222,11.777778,0.450000
2,Aaron Cresswell,ENG,DF,1989.0,West Ham,ENG-Premier League,2425.0,3836.834286,286.082540,216.569841,92.841270,0.497619
3,Aaron Hickey,SCO,DF,2002.0,Brentford,ENG-Premier League,2324.0,1859.060000,182.483333,142.694444,44.211111,0.208333
4,Aaron Lennon,ENG,MF,1987.0,Burnley,ENG-Premier League,1920.0,508.740000,88.111111,56.433333,15.966667,0.216667
...,...,...,...,...,...,...,...,...,...,...,...,...
6215,Łukasz Teodorczyk,POL,FW,1991.0,Udinese,ITA-Serie A,1920.0,213.993333,48.000000,32.311111,22.844444,0.050000
6216,Šime Vrsaljko,CRO,DF,1992.0,Atlético Madrid,ESP-La Liga,2021.0,1087.290000,119.538889,88.827778,28.211111,0.358333
6217,Žan Celar,SVN,FW,1999.0,Roma,ITA-Serie A,1819.0,0.400000,0.000000,0.000000,0.000000,0.000000
6218,Žan Majer,SVN,MF,1992.0,Lecce,ITA-Serie A,1920.0,2729.500000,254.833333,179.933333,70.633333,0.250000


In [7]:
final_df.to_csv("player_passing_weighted.csv", index=False)